In [1]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import glob
import io
from collections import namedtuple
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

### Set FilePath

In [6]:
LABEL_PATH = r"C:\Users\parzi\OneDrive - Tribhuvan University\Desktop\Minor Project\Monument Detection with CNN\Monument Object Detection\helper\Report\final.csv"
ignore_list = ['kotilingeshvara', 'til mahadev narayan temple', 'king statue']
mistake_list = ['degutale temple', 'kritipur tower']
correct_list = ['degu tale temple_KDS', 'kirtipur tower', 'degu tale']

In [7]:
POSSIBLE_LABELS = dict()

In [10]:
def csv_to_label_map_dict(path):
    df = pd.read_csv(path)
    class_list = list(df['class'])
    final_class_list = list()
    for class_name in class_list:
        if class_name not in ignore_list and class_name not in correct_list:
            if class_name in mistake_list:
                if class_name == 'degutale temple' or class_name == 'degu tale':
                    class_name = correct_list[0]
                elif class_name == 'kritipur tower':
                    class_name = correct_list[1]
            final_class_list.append(class_name)
    class_index = 1
    res_dict = {}
    for class_label in final_class_list:
        res_dict[class_label] = class_index
        class_index += 1
    return res_dict

In [11]:
POSSIBLE_LABELS = csv_to_label_map_dict(LABEL_PATH)

In [12]:
POSSIBLE_LABELS['bg'] = 0

In [15]:
POSSIBLE_LABELS

{'badrinath temple': 1,
 'basantapur tower': 2,
 'bhagavati temple': 3,
 'bhairavnath temple': 4,
 'bhaktapur tower': 5,
 'bhimeleshvara': 6,
 'bhimsen temple': 7,
 'bhupatindra malla column': 8,
 'bhuvana lakshmeshvara': 9,
 'chasin dega': 10,
 'chayasilin mandap': 11,
 'dattatreya temple': 12,
 'degu tale temple_KDS': 13,
 'fasidega temple': 14,
 'gaddi durbar': 15,
 'garud': 16,
 'golden gate': 17,
 'gopinath krishna temple': 18,
 'hanuman idol': 19,
 'indrapura': 20,
 'jagannatha temple': 21,
 'kala-bhairava': 22,
 'kasthamandap': 23,
 'kavindrapura sattal': 24,
 'kedamatha tirtha': 25,
 'kirtipur tower': 26,
 'kumari ghar': 27,
 'lalitpur tower': 28,
 'mahadev temple': 29,
 'narayan temple': 30,
 'national gallery': 31,
 'nyatapola temple': 32,
 'palace of the 55 windows': 33,
 'panchamukhi hanuman': 34,
 'pratap malla column': 35,
 'shiva temple': 36,
 'shveta bhairava': 37,
 'siddhi lakshmi temple': 38,
 'simha sattal': 39,
 'taleju bell_BDS': 40,
 'taleju bell_KDS': 41,
 'talej

In [16]:
def check_class_validity(class_name):
    if class_name == 'degutale temple' or class_name == 'degu tale':
        return 'degu tale temple_KDS'
    elif class_name == 'kritipur tower':
        return 'kirtipur tower'
    elif class_name not in list(POSSIBLE_LABELS.keys()):
        return None
    else:
        return class_name

In [17]:
class_list = []
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        all_objects = root.findall('object')
        changed_filename = xml_file.split('\\')[-1].split('.')[0] + '.JPG'
        # Handling xml parser for random background images
        if len(all_objects) != 0:
            for member in all_objects:
                class_label = check_class_validity(member[0].text)
                if class_label is not None:
                    value = (changed_filename,
                             int(root.find('size')[0].text),
                             int(root.find('size')[1].text),
                             class_label,
                             int(member[4][0].text),
                             int(member[4][1].text),
                             int(member[4][2].text),
                             int(member[4][3].text)
                             )
                    if class_label not in class_list:
                        class_list.append(class_label)
                    xml_list.append(value)
        else:
            # Setting background class to 'bg', and four bbox coordinates all to neg 1.
            value = (changed_filename,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     'bg',
                      -1,
                      -1,
                      -1,
                      -1)
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


In [18]:
# xml_to_csv(r"F:\Minor Data Collection\Final Image Data\Monument Original\Merged_Original_Augmented\test\Annotations")

In [19]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [20]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [21]:
def class_text_to_int(row_label):
    return POSSIBLE_LABELS[row_label]

In [22]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        if int(row['xmin']) > -1:
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(class_text_to_int(row['class']))
            
        else:
            xmins.append(0)
            xmaxs.append(0)
            ymins.append(0)
            ymaxs.append(0)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(0)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example

In [23]:
def TFRecord_Creator(image_dir, annotations_dir, output_file, split_type = "BDG"):
    writer = tf.io.TFRecordWriter(output_file)
    path = os.path.join(image_dir)
    examples = xml_to_csv(annotations_dir)
    #grouped by the filename so that multiple monument detection boxes are grouped in a single 'GROUP'
    grouped = split(examples, 'filename') 
    for group in tqdm(grouped):
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())
    writer.close()
    print('Successfully created the TFRecord file for ', split_type)

In [24]:
BASE_DIR = r"F:\Minor Data Collection\Final Image Data\Monument Original 512\Augmented 512 v2 (PDS included)\All Splitted"
annotations_path = 'Annotations'
jpg_images_path = 'JPEGImages'
train_dir = os.path.join(BASE_DIR, 'train')
validation_dir = os.path.join(BASE_DIR, 'validation')
test_dir = os.path.join(BASE_DIR, 'test')

In [25]:
TFRecord_Creator(os.path.join(train_dir, jpg_images_path), os.path.join(train_dir, annotations_path), os.path.join(BASE_DIR, 'train_all_512_v5.tfrecord'))

TFRecord_Creator(os.path.join(validation_dir, jpg_images_path), os.path.join(validation_dir, annotations_path), os.path.join(BASE_DIR, 'validation_all_512_v5.tfrecord'))

TFRecord_Creator(os.path.join(test_dir, jpg_images_path), os.path.join(test_dir, annotations_path), os.path.join(BASE_DIR, 'test_all_512_v5.tfrecord'))

100%|████████████████████████████████████████████████████████████████████████████| 19609/19609 [03:34<00:00, 91.45it/s]


Successfully created the TFRecord file for  BDG


100%|██████████████████████████████████████████████████████████████████████████████| 2551/2551 [00:32<00:00, 78.59it/s]


Successfully created the TFRecord file for  BDG


100%|██████████████████████████████████████████████████████████████████████████████| 2357/2357 [00:26<00:00, 87.60it/s]


Successfully created the TFRecord file for  BDG


In [74]:
# image_dir = './Dataset/test/JPEGImages'
# annotations_dir = './Dataset/test/Annotations'
# output_file = './monument-test.tfrecord'
# main(image_dir, annotations_dir, output_file)

In [ ]:
# def xml_to_csv(path):
#     xml_list = []
#     for xml_file in glob.glob(path + '/*.xml'):
#         tree = ET.parse(xml_file)
#         root = tree.getroot()
#         for member in root.findall('object'):
#             class_label = check_class_validity(member[0].text)
#             if class_label is not None:
#                 value = (root.find('filename').text,
#                          int(root.find('size')[0].text),
#                          int(root.find('size')[1].text),
#                          class_label,
#                          int(member[4][0].text),
#                          int(member[4][1].text),
#                          int(member[4][2].text),
#                          int(member[4][3].text)
#                          )
#                 xml_list.append(value)
#     column_name = ['filename', 'width', 'height',
#                    'class', 'xmin', 'ymin', 'xmax', 'ymax']
#     xml_df = pd.DataFrame(xml_list, columns=column_name)
#     return xml_df